配置和启动 PySpark：

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
# 本地模式
spark = SparkSession.builder.master("local[*]").appName("Reading Text").getOrCreate()
sc = spark.sparkContext
# sc.setLogLevel("ERROR")
print(spark)
print(sc)

<SparkContext master=local[*] appName=Reading Text>


查看环境变量：

In [2]:
import os

print(os.getenv("HADOOP_HOME"))
print(os.getenv("SPARK_HOME"))

!pyspark --version

E:\software\hadoop
E:\software\spark


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.3.2
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 11.0.18
Branch HEAD
Compiled by user liangchi on 2023-02-10T19:57:40Z
Revision 5103e00c4ce5fcc4264ca9c4df12295d42557af6
Url https://github.com/apache/spark
Type --help for more information.


利用 PySpark 读取文本文件。注意此时只是创建了一个对文件的描述，文件内容并没有加载到内存中。

In [3]:
file = sc.textFile("data/UNv1.0.en-zh.zh")

In [5]:
print(file)

data/UNv1.0.en-zh.zh MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0


查看文件的行数：

In [4]:
print(file.count())

15886041


查看前5行：

In [5]:
text1 = file.take(5)
print(*enumerate(text1), sep="\n")

(0, '第918(1994)号决议')
(1, '1994年5月17日安全理事会第3377次会议通过')
(2, '安全理事会，')
(3, '重申其以往关于卢旺达局势的所有决议，特别是成立联合国卢旺达援助团(联卢援助团)的1993年10月5日第872(1993)号决议，延长联卢援助团任务期限至1994年7月29日的1994年4月5日第909(1994)号决议，以及调整联卢援助团的任务规定的1994年4月21日第912(1994)号决议，')
(4, '回顾安理会主席以安理会名义在1994年4月7日发表的声明(S/PRST/ 1994/16)和在1994年4月30日发表的声明(S/PRST/1994/21)，')


查看随机抽取的10行（可能会比较慢！访问 http://localhost:4040 查看进度）：

In [6]:
text2 = file.takeSample(withReplacement=False, num=10, seed=123)
print(*enumerate(text2), sep="\n")

(0, '第八编. 共同支助事务')
(1, '14. 在提出这一建议的同时，我必须提请注意观察员部队经费短缺问题。')
(2, '报告期间内，在国家政策、立法和做法方面是否发生了不利于第十三条所载权利的变化？')
(3, '议程项目39')
(4, '三. 审议缔约国根据《公约》第九条提交的报告、评论和资料 25-40 9')
(5, '第十七届会议')
(6, '4. 2004年9月21日至25日，特别报告员在联合国人权事务高级专员办事处(人权高专办)简要介绍了情况。')
(7, 'IMF')
(8, '因此，有必要调整观察员部队的核定兵力，增加10名军事特遣队人员。')
(9, '《民法》中，组织、协会、企业、基金会及其他具有私营性质的实体被定义为私营法人。')


利用 Filter 操作筛选符合条件的行：

In [7]:
# 所有包含“乌克兰”的行
ukraine = file.filter(lambda x: x.__contains__("乌克兰"))

In [8]:
print(ukraine)

PythonRDD[3] at RDD at PythonRDD.scala:53


注意，运行上述语句并不会直接进行计算，而只是把要进行的操作保存了下来。只有当真正需要获取结果时计算才会发生，比如下面获取前5行的操作：

In [8]:
text3 = ukraine.take(5)
print(*enumerate(text3), sep="\n")

(0, '根据乌克兰和格鲁吉亚共和国政府的请求，美国航天局与空间雷达实验室 2号协同工作的和科学家于1994年10月得到了有关航天飞机的数据，将这些数据转送了两国。')
(1, '这个机构使用卫星图象、作物模型以及遥感气候数据支持国务院对前苏联各国的粮食需要，特别是遭受干旱的乌克兰作出的评价。')
(2, '最近签订的协议包括1994年6月30日与俄罗斯联邦签订的协议和1994年9月16日与乌克兰签订的协议。')
(3, '白俄罗斯的种族比例如下:白俄罗斯人,78%;俄罗斯人,13%;波兰人,4%;乌克兰人,3%;犹太人1%;其它民族,1%。')
(4, '乌克兰 第十三次报告 1994年1月5日 -')


Filter 操作的参数是一个函数，该函数输入一个字符串，输出 `True` 或 `False` 的取值。上面该函数通过 lambda 表达式实现，也可以使用定义好的函数：

In [9]:
def my_filter(x):
    return x.__contains__("乌克兰") and x.__contains__("俄罗斯")

ukraine2 = file.filter(my_filter)
text4 = ukraine2.take(5)
print(*enumerate(text4), sep="\n")

(0, '最近签订的协议包括1994年6月30日与俄罗斯联邦签订的协议和1994年9月16日与乌克兰签订的协议。')
(1, '白俄罗斯的种族比例如下:白俄罗斯人,78%;俄罗斯人,13%;波兰人,4%;乌克兰人,3%;犹太人1%;其它民族,1%。')
(2, '为了根据俄罗斯和乌克兰的本国及国际方案对地球表面进行观测，包括对自然环境进行环境监测，并且为了获得北极地区冰帽状况以及太平洋暗礁地带状况的最新情报，于1995年8月31日发射了一颗Sich-1号卫星（乌克兰）。')
(3, '该空间站装备的科学仪器是由以下国家的科学家和专家设计的：奥地利、比利时、加拿大、古巴、捷克共和国、芬兰、法国、德国、希腊、匈牙利、意大利、吉尔吉斯斯坦、波兰、罗马尼亚、俄罗斯联邦、斯洛伐克、瑞典、乌克兰、大不列颠及北爱尔兰联合王国、乌兹别克斯坦以及欧洲航天局的成员国。')
(4, '该航天器的初期轨道，其近地点为500公里，远地点为300,000公里，在预期三年的飞行期限间，达到运行轨道的稳定标准和从俄罗斯联邦与乌克兰地面站基本连续可见度标准，达到从发射后一年近地点高度迅速变换达到40,000公里的高度。')


Map 操作可以对数据的每一行进行变换，例如将字符串分割成若干字串的列表：

In [10]:
split = file.map(lambda x: x.split("，"))
text5 = split.take(5)
print(*enumerate(text5), sep="\n")

(0, ['第918(1994)号决议'])
(1, ['1994年5月17日安全理事会第3377次会议通过'])
(2, ['安全理事会', ''])
(3, ['重申其以往关于卢旺达局势的所有决议', '特别是成立联合国卢旺达援助团(联卢援助团)的1993年10月5日第872(1993)号决议', '延长联卢援助团任务期限至1994年7月29日的1994年4月5日第909(1994)号决议', '以及调整联卢援助团的任务规定的1994年4月21日第912(1994)号决议', ''])
(4, ['回顾安理会主席以安理会名义在1994年4月7日发表的声明(S/PRST/ 1994/16)和在1994年4月30日发表的声明(S/PRST/1994/21)', ''])


Flat map 可以把上述列表展开：

In [11]:
split2 = file.flatMap(lambda x: x.split("，"))
text6 = split2.take(10)
print(*enumerate(text6), sep="\n")

(0, '第918(1994)号决议')
(1, '1994年5月17日安全理事会第3377次会议通过')
(2, '安全理事会')
(3, '')
(4, '重申其以往关于卢旺达局势的所有决议')
(5, '特别是成立联合国卢旺达援助团(联卢援助团)的1993年10月5日第872(1993)号决议')
(6, '延长联卢援助团任务期限至1994年7月29日的1994年4月5日第909(1994)号决议')
(7, '以及调整联卢援助团的任务规定的1994年4月21日第912(1994)号决议')
(8, '')
(9, '回顾安理会主席以安理会名义在1994年4月7日发表的声明(S/PRST/ 1994/16)和在1994年4月30日发表的声明(S/PRST/1994/21)')


关闭 Spark：

In [12]:
sc.stop()